# Mesh refinement study

## COMSOL files 01.MR.lam.001 - 003

- /Users/hkromer/02_PhD/02_Data/01_COMSOL/cfd/01.MR/01.MR.lam.001.mph

The objective is to find out if the temperature properties change with the mesh

- 2 mm FWHM spot
- 200 mm domain length
- 40 mm outer diameter of target
- 2 mm coolant channel
- 3 mm CuOFE layer
- 5 e-6 m Ti layer
- 3 l/min flow arte water
- 300 W beam power
- 200 rpm rotational speed
- 20 degC water inlet temperature
- laminar flow

<img src="001.MR.lam.001.meshplot.png" style="width: 10%">
<img src="001.MR.lam.002.meshplot.png" style="width: 10%">
<img src="001.MR.lam.003.meshplot.png" style="width: 10%">

In [ ]:
import numpy as np
import os
import re
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from scipy import interpolate

In [ ]:
os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2018/bin/x86_64-darwin'
plt.rc('text', usetex=True)
plt.rc('font', weight='bold')
matplotlib.rcParams['mathtext.fontset'] = 'custom'
matplotlib.rcParams['mathtext.rm'] = 'Arial'
matplotlib.rcParams['mathtext.it'] = 'Arial:italic'
matplotlib.rcParams['mathtext.bf'] = 'Arial:bold'
matplotlib.rcParams['mathtext.tt'] = 'Arial'
matplotlib.rcParams['mathtext.cal'] = 'Arial'
matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

In [ ]:
folder = '/Users/hkromer/02_PhD/02_Data/01_COMSOL/cfd/01.MR/DataAlongZatBeamspot/' 
output_figures = '/Users/hkromer/polybox/Thesis/Chapters/RotatingIonBeamTarget/Figures/mesh_refinement/'

In [ ]:
files = [s for s in os.listdir(folder) if s.endswith('csv')]

In [ ]:
lst_df = []
for file in files:
    df = pd.read_csv(f'{folder}/{file}', skiprows=8)
    df.columns = ['x', 'y', 'z', 'T', 'U']   # x in mm, y in mm, z in mm, T in K, U in m/s
    df['T_degC'] = df['T'] - 273.15
    run = re.findall(r'lam\.(\d\d\d)\.', file)[0]
    df['run'] = run
    if run == '001':
        df['mesh_cat'] = 'A'
        df['mesh_cat'] = df['mesh_cat'].astype('category')
    if run == '002':
        df['mesh_cat'] = 'B'
        df['mesh_cat'] = df['mesh_cat'].astype('category')
    if run == '003':
        df['mesh_cat'] = 'C'
        df['mesh_cat'] = df['mesh_cat'].astype('category')
    lst_df.append(df)
    
df_all = pd.concat(lst_df)
df_all.info()

In [ ]:
# reference dataframe is the 003, most finest mesh
ref_df = df_all[ df_all['run'] == '003' ]
df_comp = df_all[ df_all['run'] != '003' ]

In [ ]:
# sanity check
print(len(df_all))
print(len(df_comp))
print(len(ref_df))

## Relative difference in temperature

In [ ]:
runs = df_comp['run'].unique()
mesh_cats = df_comp['mesh_cat'].unique()
ref_df = ref_df.sort_values(by=['z'])
z0 = ref_df['z'].values  # mesh points of the finest mesh
T0 = ref_df['T'].values  # T of the finest mesh

colors = ['darkblue', 'darkorange']
markers = ['o', 's']
fs = 28
ls = 28
msize = 10

plt.figure(figsize=(10,8))
for r, c, m, lbl  in zip(runs, colors, markers, mesh_cats):
    this_df = df_comp[ df_comp['run'] == r ]
    this_df = this_df.sort_values(by=['z'])
    # interpolate the T
    interpT = interpolate.interp1d(this_df['z'].values, this_df['T'].values, fill_value='extrapolate')
    
    # extract T at the positions of the finest mesh
    T = interpT(z0)
    
    # calculate difference in T 
    rel_diff_T = 100.0*np.abs(T-T0)/T0
    
    # plot
    plt.plot(z0, rel_diff_T, c=c, label=f'{lbl}', marker=m, markersize=msize)
#     plt.scatter(z0, rel_diff_T, c=c, marker=m, s=msize)

ax = plt.gca()
ylim = ax.get_ylim()
plt.plot([-17, -17], [-10, 10], color='black', linestyle='--', linewidth=3)
plt.plot([-20, -20], [-10, 10], color='black', linestyle='--', linewidth=3)
plt.plot([-15, -15], [-10, 10], color='black', linestyle='--', linewidth=3)
plt.ylim(ylim)
ax.tick_params(axis = 'both', which = 'major', labelsize = ls)
ax.text(-17.5, 1.15, r'\textbf{Cu}', color='black', fontsize=fs)
ax.text(-16.75, 1.15, r'\textbf{Water}', color='black', fontsize=fs)
# minor ticks x
minor_locator = AutoMinorLocator(2)
ax.xaxis.set_minor_locator(minor_locator)
# minor ticks y
minor_locator = AutoMinorLocator(2)
ax.yaxis.set_minor_locator(minor_locator)
handles,labels = ax.get_legend_handles_labels()
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
plt.xlabel(r'\textbf{z [mm]}', fontsize=fs)
plt.ylabel(r'\textbf{Relative difference in temperature [\%]}', fontsize=fs)
legend = plt.legend(title=r'\textbf{Mesh}', fontsize=fs, loc=(0.72,0.67), framealpha=1.0)
plt.setp(legend.get_title(),fontsize=fs)
plt.ylim(-0.2,2.5)
# plt.title('Mesh refinement')
# plt.savefig(f'{folder}/mesh_refinement_T.png', dpi=600)
plt.savefig(f'{output_figures}/mesh_refinement_T.pdf', dpi=600)
plt.show()
print(ylim)

## Relative difference in velocity magnitude


In [ ]:
runs = df_comp['run'].unique()
mesh_cats = df_comp['mesh_cat'].unique()
ref_df = ref_df.sort_values(by=['z'])
z0 = ref_df['z'].values  # mesh points of the finest mesh
U0 = ref_df['U'].values  # T of the finest mesh

colors = ['darkblue', 'darkorange']
markers = ['o', 's']

msize = 10

plt.figure(figsize=(10,8))
for r, c, m, lbl  in zip(runs, colors, markers, mesh_cats):
    this_df = df_comp[ df_comp['run'] == r ]
    this_df = this_df.sort_values(by=['z'])
    # interpolate the T
    interpU = interpolate.interp1d(this_df['z'].values, this_df['U'].values, fill_value='extrapolate')
    
    # extract U at the positions of the finest mesh
    U = interpU(z0)
    
    # calculate difference in U 
    rel_diff_U = 100.0*np.abs(U-U0)/U0
    
    # plot
    plt.plot(z0, rel_diff_U, c=c, label=f'{lbl}', marker=m, markersize=msize)
#     plt.scatter(z0, rel_diff_U, c=c, marker=m, s=msize)

ax = plt.gca()
ylim = ax.get_ylim()
plt.plot([-17, -17], [-10, 10], color='black', linestyle='--', linewidth=3)
plt.plot([-20, -20], [-10, 10], color='black', linestyle='--', linewidth=3)
plt.plot([-15, -15], [-10, 10], color='black', linestyle='--', linewidth=3)
plt.ylim(-0.14, 8)
plt.xlim(-17.2, -14.8)
ax.tick_params(axis = 'both', which = 'major', labelsize = ls)

# minor ticks x
minor_locator = AutoMinorLocator(2)
ax.xaxis.set_minor_locator(minor_locator)
# minor ticks y
minor_locator = AutoMinorLocator(2)
ax.yaxis.set_minor_locator(minor_locator)
handles,labels = ax.get_legend_handles_labels()
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
plt.xlabel(r'\textbf{z [mm]}', fontsize=fs)
plt.ylabel(r'\textbf{Relative difference}'+ '\n'+ r'\textbf{in velocity magnitude [\%]}', fontsize=fs)
legend = plt.legend(title=r'\textbf{Mesh}', fontsize=fs, loc=(0.68,0.67), framealpha=1)
plt.setp(legend.get_title(),fontsize=fs)
plt.ylim(-0.5,8)
# plt.title('Mesh refinement')
# plt.savefig(f'{folder}/mesh_refinement_U.png', dpi=600)
plt.savefig(f'{output_figures}/mesh_refinement_U.pdf', dpi=600)
plt.show()

## Plot temperature distribution

In [ ]:
runs = df_all['run'].unique().tolist()
runs = sorted(runs)

mesh_cats = df_all['mesh_cat'].unique()

colors = ['darkblue', 'darkred', 'darkorange']
markers = ['o', 'p', 's']

msize = 10

plt.figure(figsize=(10,8))
for r, c, m, lbl  in zip(runs, colors, markers, mesh_cats):
    this_df = df_all[ df_all['run'] == r ]
    this_df = this_df.sort_values(by=['z'])
    
    # plot
#     plt.scatter(this_df['z'], this_df['T']-273.15, c=c, marker=m, label=None, s=msize)
    plt.plot(this_df['z'], this_df['T']-273.15, c=c, label=f'{lbl}', marker=m, markersize=msize)

ax = plt.gca()
ylim = ax.get_ylim()
plt.plot([-17, -17], [-10, 1000], color='black', linestyle='--', linewidth=3)
plt.plot([-20, -20], [-10, 1000], color='black', linestyle='--', linewidth=3)
plt.plot([-15, -15], [-10, 1000], color='black', linestyle='--', linewidth=3)
plt.ylim(ylim[0], 140)
ax.tick_params(axis = 'both', which = 'major', labelsize = ls)
ax.text(-17.5, 365-273.15-20, r'\textbf{Cu}', color='black', fontsize=fs)
ax.text(-16.85, 365-273.15-20, r'\textbf{Water}', color='black', fontsize=fs)
plt.grid()
plt.xlabel(r'\textbf{z [mm]}', fontsize=fs)
plt.ylabel(r'\textbf{Temperature [$^\circ$C]}', fontsize=fs)
# minor ticks x
minor_locator = AutoMinorLocator(2)
ax.xaxis.set_minor_locator(minor_locator)
# minor ticks y
minor_locator = AutoMinorLocator(2)
ax.yaxis.set_minor_locator(minor_locator)
handles,labels = ax.get_legend_handles_labels()

handles = [handles[0], handles[2], handles[1]]
labels = [labels[0], labels[2], labels[1]]
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
leg = ax.legend(handles,labels,title=r'\textbf{Mesh}',loc=(0.78,0.59), fontsize=fs, framealpha=1.0)
leg.set_title(r'\textbf{Mesh}', prop = {'size':fs})

plt.setp(legend.get_title(),fontsize=fs)
# plt.title('Mesh refinement')
# plt.savefig(f'{folder}/mesh_refinement_T_abs.png', dpi=600)
plt.savefig(f'{output_figures}/mesh_refinement_T_abs.pdf', dpi=600)
plt.show()
print(ylim)

## Plot absolute velocity magnitude

In [ ]:
runs = df_all['run'].unique().tolist()
runs = sorted(runs)

mesh_cats = df_all['mesh_cat'].unique()

colors = ['darkblue', 'darkred', 'darkorange']
markers = ['o', 'p', 's']

msize = 10

plt.figure(figsize=(10,8))
for r, c, m, lbl  in zip(runs, colors, markers, mesh_cats):
    this_df = df_all[ df_all['run'] == r ]
    this_df = this_df.sort_values(by=['z'])
    
    # plot
#     plt.scatter(this_df['z'], this_df['U'], c=c, marker=m, label=None, s=msize)
    plt.plot(this_df['z'], this_df['U'], c=c, label=f'{lbl}', marker=m, markersize=msize)

ax = plt.gca()
ylim = ax.get_ylim()
plt.plot([-17, -17], [-10, 10], color='black', linestyle='--', linewidth=3)
plt.plot([-20, -20], [-10, 10], color='black', linestyle='--', linewidth=3)
plt.plot([-15, -15], [-10, 10], color='black', linestyle='--', linewidth=3)
plt.ylim(ylim)
plt.xlim(-17.2, -14.8)
ax.tick_params(axis = 'both', which = 'major', labelsize = ls)

plt.grid()
plt.xlabel(r'\textbf{z [mm]}', fontsize=fs)
plt.ylabel(r'\textbf{Velocity magnitude [m/s]}', fontsize=fs)
handles,labels = ax.get_legend_handles_labels()

handles = [handles[0], handles[2], handles[1]]
labels = [labels[0], labels[2], labels[1]]
ylims = plt.gca().get_ylim()
plt.ylim(ylims[0],0.4)

# minor ticks x
minor_locator = AutoMinorLocator(2)
ax.xaxis.set_minor_locator(minor_locator)
# minor ticks y
minor_locator = AutoMinorLocator(2)
ax.yaxis.set_minor_locator(minor_locator)

ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
leg = ax.legend(handles,labels,title=r'\textbf{Mesh}',loc=(0.78,0.59), fontsize=fs, framealpha=1)
leg.set_title(r'\textbf{Mesh}', prop = {'size':fs})

# plt.title('Mesh refinement')
# plt.savefig(f'{folder}/mesh_refinement_U_abs.png', dpi=600)
plt.savefig(f'{output_figures}/mesh_refinement_U_abs.pdf', dpi=600)
plt.show()